In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator,Example,Dataset

import spacy
import numpy as np

import random
import math
import time
import en_core_web_sm
import pandas as pd


In [12]:
!pip uninstall -y spacy
!pip install -U spacy 

     |████████████████████████████████| 12.8MB 14.6MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 9.1MB 36.5MB/s 
     |████████████████████████████████| 460kB 51.2MB/s 
     |████████████████████████████████| 624kB 39.6MB/s 
     |████████████████████████████████| 122kB 49.9MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107107 sha256=dee68462e0cf975c449973429c6299faeea129ae52e88c034f74923742acdc38
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: smart-open 5.1.0
    Uninstalling smart-open-5.1.0:
      Successfully uninstalled smart-open-5.1.0
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
s08 = pd.read_csv("/content/drive/MyDrive/END2.0/Question_Answer_Dataset_v1.2/S08/question_answer_pairs.txt", sep = '\t',encoding = "ISO-8859-1")
s09 = pd.read_csv("/content/drive/MyDrive/END2.0/Question_Answer_Dataset_v1.2/S09/question_answer_pairs.txt", sep = '\t',encoding = "ISO-8859-1")
s10 = pd.read_csv("/content/drive/MyDrive/END2.0/Question_Answer_Dataset_v1.2/S10/question_answer_pairs.txt", sep = '\t',encoding = "ISO-8859-1")

In [4]:
s08.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,data/set3/a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,data/set3/a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,data/set3/a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,data/set3/a4
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,data/set3/a4


In [5]:
s09.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Volta an Italian physicist?,yes,easy,easy,data/set4/a10
1,Alessandro_Volta,Was Volta an Italian physicist?,yes,easy,easy,data/set4/a10
2,Alessandro_Volta,Is Volta buried in the city of Pittsburgh?,no,easy,easy,data/set4/a10
3,Alessandro_Volta,Is Volta buried in the city of Pittsburgh?,no,easy,easy,data/set4/a10
4,Alessandro_Volta,Did Volta have a passion for the study of elec...,yes,easy,medium,data/set4/a10


In [6]:
s10.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,Alessandro Volta was not a professor of chemis...,easy,easy,data/set4/a10
1,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,No,easy,hard,data/set4/a10
2,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Alessandro Volta did invent the remotely opera...,easy,easy,data/set4/a10
3,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Yes,easy,easy,data/set4/a10
4,Alessandro_Volta,Was Alessandro Volta taught in public schools?,Volta was taught in public schools.,easy,easy,data/set4/a10


In [23]:
df_qa = s08.append(s09).append(s10).reset_index()

In [24]:
assert (df_qa.shape[0] == (s08.shape[0] + s09.shape[0] + s10.shape[0]))

In [25]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [11]:
%%bash
python -m spacy download en
# python -m spacy download de 

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-06-18 05:53:29.415806: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [12]:
spacy_en = spacy.load('en_core_web_sm')
# spacy_de = spacy.load('de_core_news_sm')


In [26]:
df_qa.shape

(3998, 7)

In [27]:
df_qa.head()

,index,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,data/set3/a4
1,1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,data/set3/a4
2,2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,data/set3/a4
3,3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,data/set3/a4
4,4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,data/set3/a4


In [33]:
df_qa.Answer[0]

'yes'

In [29]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [30]:
Question = Field(tokenize = tokenize_en, 
            sequential = True, 
            include_lengths=True,
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

Answer = Field(tokenize = tokenize_en, 
            sequential = True, 
            include_lengths=True,
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [31]:
fields = [('Question', Question), ('Answer', Answer)]

In [37]:
example = [Example.fromlist([str(df_qa.Question[i]),str(df_qa.Answer[i])], fields) for i in range(df_qa.shape[0])] 

In [38]:
QnADataset = Dataset(example, fields)

In [39]:
(train_data, valid_data) = QnADataset.split(split_ratio=[70, 30], random_state = random.seed(SEED))

In [42]:
vars(train_data[0])

{'Answer': ['nan'],
 'Question': ['what',
  'instrument',
  'was',
  'produced',
  'after',
  'the',
  'xylophone',
  'in',
  'the',
  '1920s',
  '?']}

In [43]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
# print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 2799
Number of validation examples: 1199


In [45]:
vars(valid_data.examples[0])

{'Answer': ['no'],
 'Question': ['can',
  'turtles',
  'spend',
  'all',
  'their',
  'time',
  'underwater',
  '?']}

In [47]:
Question.build_vocab(train_data, min_freq = 2)
Answer.build_vocab(train_data, min_freq = 2)

In [48]:
print(f"Unique tokens in source (de) vocabulary: {len(Question.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(Answer.vocab)}")

Unique tokens in source (de) vocabulary: 2251
Unique tokens in target (en) vocabulary: 1302


In [112]:
import pickle
with open('/content/drive/MyDrive/END2.0/session_7_question_tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Question.vocab.stoi, tokens)
with open('/content/drive/MyDrive/END2.0/session_7_answer_tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Answer.vocab.stoi, tokens)

In [49]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [67]:
BATCH_SIZE = 128

train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort_key = lambda x : len(x.Question))

In [68]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [69]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        # print("-INPUT-"*20)
        # print(input.shape)
        # print("-"*20)
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [70]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        # print(trg)
        batch_size = trg.shape[1]
        # print("-SEQ-2-SEQ SHAPES-"*20)
        # print(src.shape ,trg.shape)
        # print(batch_size)
        # print("-"*20)
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # print(hidden.shape)
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [71]:
INPUT_DIM = len(Question.vocab)
OUTPUT_DIM = len(Answer.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [72]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2251, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1302, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=1302, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [73]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,933,910 trainable parameters


In [74]:
optimizer = optim.Adam(model.parameters())

In [75]:
TRG_PAD_IDX = Answer.vocab.stoi[Answer.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [76]:

def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src = batch.Question[0]
        trg = batch.Answer[0]
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [77]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.Question[0]
            trg = batch.Answer[0]

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [78]:

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [79]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 5.022 | Train PPL: 151.724
	 Val. Loss: 3.630 |  Val. PPL:  37.712
Epoch: 02 | Time: 0m 2s
	Train Loss: 4.295 | Train PPL:  73.346
	 Val. Loss: 3.575 |  Val. PPL:  35.706
Epoch: 03 | Time: 0m 2s
	Train Loss: 4.229 | Train PPL:  68.666
	 Val. Loss: 3.570 |  Val. PPL:  35.512
Epoch: 04 | Time: 0m 2s
	Train Loss: 4.157 | Train PPL:  63.885
	 Val. Loss: 3.617 |  Val. PPL:  37.220
Epoch: 05 | Time: 0m 2s
	Train Loss: 4.105 | Train PPL:  60.623
	 Val. Loss: 3.665 |  Val. PPL:  39.072
Epoch: 06 | Time: 0m 2s
	Train Loss: 4.074 | Train PPL:  58.766
	 Val. Loss: 4.980 |  Val. PPL: 145.486
Epoch: 07 | Time: 0m 2s
	Train Loss: 4.007 | Train PPL:  54.978
	 Val. Loss: 3.695 |  Val. PPL:  40.228
Epoch: 08 | Time: 0m 2s
	Train Loss: 3.966 | Train PPL:  52.773
	 Val. Loss: 3.620 |  Val. PPL:  37.340
Epoch: 09 | Time: 0m 2s
	Train Loss: 3.944 | Train PPL:  51.621
	 Val. Loss: 3.609 |  Val. PPL:  36.923
Epoch: 10 | Time: 0m 2s
	Train Loss: 3.910 | Train PPL:  49.881


In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

In [81]:
vars(valid_data[0])

{'Answer': ['no'],
 'Question': ['can',
  'turtles',
  'spend',
  'all',
  'their',
  'time',
  'underwater',
  '?']}

In [97]:
vars(valid_data[0])

{'Answer': ['no'],
 'Question': ['can',
  'turtles',
  'spend',
  'all',
  'their',
  'time',
  'underwater',
  '?']}

In [98]:
BATCH_SIZE = 1

valid_iterator = BucketIterator.splits(
    (valid_data[0:10]), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort_key = lambda x : len(x.Question))

In [132]:
path='/content/tut1-model.pt'
model.load_state_dict(torch.load(path));
model.eval()

question ="hello this is pralay"
answer = "this is pralay"
tokenizer_file = open('/content/drive/MyDrive/END2.0/session_7_question_tokenizer.pkl', 'rb')
question_tokenizer = pickle.load(tokenizer_file)
tokenizer_file = open('/content/drive/MyDrive/END2.0/session_7_answer_tokenizer.pkl', 'rb')
answer_tokenizer = pickle.load(tokenizer_file)


tokenized = tokenize_en(text)
# convert to integer sequence using predefined tokenizer dictionary
question_indexed = [question_tokenizer[t] for t in tokenized]        
# compute no. of words        
length = [len(question_indexed)]
# convert to tensor                                    
tensor = torch.LongTensor(question_indexed).to(device)   

question_tensor = tensor.unsqueeze(1).T  


tokenized = tokenize_en(text)
# convert to integer sequence using predefined tokenizer dictionary
answer_indexed = [answer_tokenizer[t] for t in tokenized]        
# compute no. of words        
length = [len(answer_indexed)]
# convert to tensor                                    
tensor = torch.LongTensor(answer_indexed).to(device)   

answer_tensor = tensor.unsqueeze(1).T  


In [133]:
output = model(question_tensor, answer_tensor, 0)
print(output)
print(output.shape)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')
torch.Size([1, 4, 1302])
